# Step 3: Combining the Datasets

In [1]:
# imports
import pandas as pd
import numpy as np

**Begin by merging the data we got from the APIs with the population data for each state**

First, we import the files

In [2]:
# df_quality = pd.read_csv('Article_data_with_RevIDs_Scores.csv')
df_quality = pd.read_csv('1K_Article_data_with_RevIDs_Scores.csv')
df_quality.head()

revision_id           page_title    state article_quality  \
0       104730   Abbeville, Alabama  Alabama            Stub   
1       104761  Adamsville, Alabama  Alabama            Stub   
2       105188     Addison, Alabama  Alabama            Stub   
3       104726       Akron, Alabama  Alabama            Stub   
4       105109   Alabaster, Alabama  Alabama            Stub   

                                                 url  
0   https://en.wikipedia.org/wiki/Abbeville,_Alabama  
1  https://en.wikipedia.org/wiki/Adamsville,_Alabama  
2     https://en.wikipedia.org/wiki/Addison,_Alabama  
3       https://en.wikipedia.org/wiki/Akron,_Alabama  
4   https://en.wikipedia.org/wiki/Alabaster,_Alabama

The NST population excel file is quite messy and not properly formatted. Hence, I specify an unusual row for the header and drop the first 5 rows of the data (because they are not states).

In [3]:
df_NST_pop = pd.read_excel('NST-EST2022-POP.xlsx', header=3)
df_NST_pop = df_NST_pop.iloc[5:, :]

# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html
# Source: https://stackoverflow.com/questions/16167829/in-pandas-how-can-i-reset-index-without-adding-a-new-column
df_NST_pop.reset_index(drop=True, inplace=True)
df_NST_pop.head()

Unnamed: 0  Unnamed: 1        2020        2021        2022
0     .Alabama   5024356.0   5031362.0   5049846.0   5074296.0
1      .Alaska    733378.0    732923.0    734182.0    733583.0
2     .Arizona   7151507.0   7179943.0   7264877.0   7359197.0
3    .Arkansas   3011555.0   3014195.0   3028122.0   3045637.0
4  .California  39538245.0  39501653.0  39142991.0  39029342.0

Now, from the assignment instructions, we can infer that we only need to concern ourselves with the state column and the 2022 population data, so let's drop some columns and do some labeling. Also, notice that for some reason all the state values begin with a '.' for some reason, so let's get rid of those.

In [4]:
# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
df_NST_pop.drop(['Unnamed: 1', 2020, 2021], axis=1, inplace=True)

# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
# Source: https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/
df_NST_pop.rename(columns={'Unnamed: 0': 'state', 2022 : 'population'}, inplace=True)

# I'm not really sure how this works, but I got the following approach from
# https://stackoverflow.com/questions/42349572/remove-first-x-number-of-characters-from-each-row-in-a-column-of-a-python-datafr
df_NST_pop['state'] = df_NST_pop['state'].str[1:]

df_NST_pop.head()

state  population
0     Alabama   5074296.0
1      Alaska    733583.0
2     Arizona   7359197.0
3    Arkansas   3045637.0
4  California  39029342.0

Now we can merge the two tables and rename the 'page_title' column to 'article_title' to match the schema in the assignment instructions

In [5]:
# Source: https://towardsdatascience.com/left-join-with-pandas-data-frames-in-python-c29c85089ba4
# Source: https://stackoverflow.com/questions/53645882/pandas-merging-101
df_quality_pop = df_quality.merge(df_NST_pop, on='state', how='inner')

# Rearrange the column order
df_quality_pop = df_quality_pop[['revision_id', 'page_title', 'state', 'population', 'article_quality', 'url']]

# Rename the 'page_title' column
df_quality_pop.rename(columns = {'page_title': 'article_title'}, inplace=True)

df_quality_pop.head()

revision_id        article_title    state  population article_quality  \
0       104730   Abbeville, Alabama  Alabama   5074296.0            Stub   
1       104761  Adamsville, Alabama  Alabama   5074296.0            Stub   
2       105188     Addison, Alabama  Alabama   5074296.0            Stub   
3       104726       Akron, Alabama  Alabama   5074296.0            Stub   
4       105109   Alabaster, Alabama  Alabama   5074296.0            Stub   

                                                 url  
0   https://en.wikipedia.org/wiki/Abbeville,_Alabama  
1  https://en.wikipedia.org/wiki/Adamsville,_Alabama  
2     https://en.wikipedia.org/wiki/Addison,_Alabama  
3       https://en.wikipedia.org/wiki/Akron,_Alabama  
4   https://en.wikipedia.org/wiki/Alabaster,_Alabama

**Now, let's merge in the geographic region corresponding to each state:**

First, let's import the geographical data. According to a classmate, she was told to focus on the 'DIVISION' column - so we can ignore the 'REGION' column. Let's also rename the columns

In [6]:
df_region = pd.read_excel('US States by Region - US Census Bureau.xlsx')

# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
df_region.drop(['REGION'], axis=1, inplace=True)

# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
# Source: https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/
df_region.rename(columns={'DIVISION': 'regional_division', 'STATE' : 'state'}, inplace=True)

df_region.head(10)

regional_division          state
0               NaN            NaN
1       New England            NaN
2               NaN    Connecticut
3               NaN          Maine
4               NaN  Massachusetts
5               NaN  New Hampshire
6               NaN   Rhode Island
7               NaN        Vermont
8   Middle Atlantic            NaN
9               NaN     New Jersey

Now, we need to fill in the DIVISION column so that it has no missing values, to make the merge easier

In [7]:
# Source: https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
# Source: used this site to learn that .notna() is not a function for floats and what to do instead
# https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-a-certain-column-is-nan
# Source: https://pandas.pydata.org/docs/reference/api/pandas.notna.html
current_division = np.nan

for index, row in df_region.iterrows():
    if pd.notna(row.regional_division):
        current_division = row.regional_division
    
    else:
        row.regional_division = current_division

df_region.head(10)

regional_division          state
0               NaN            NaN
1       New England            NaN
2       New England    Connecticut
3       New England          Maine
4       New England  Massachusetts
5       New England  New Hampshire
6       New England   Rhode Island
7       New England        Vermont
8   Middle Atlantic            NaN
9   Middle Atlantic     New Jersey

Next, we can drop the null values

In [8]:
df_region.dropna(inplace=True)
df_region.head()

regional_division          state
2       New England    Connecticut
3       New England          Maine
4       New England  Massachusetts
5       New England  New Hampshire
6       New England   Rhode Island

Finally, we can merge this dataset with the other dataframe we have and rearrange the columns to match the order they appear in the assignment schema

In [9]:
# Source: https://towardsdatascience.com/left-join-with-pandas-data-frames-in-python-c29c85089ba4
# Source: https://stackoverflow.com/questions/53645882/pandas-merging-101
df_quality_pop_division = df_quality_pop.merge(df_region, on='state', how='inner')

df_quality_pop_division = df_quality_pop_division[['state', 'regional_division', 'population', 'article_title', 
                                                   'revision_id', 'article_quality']]
df_quality_pop_division.head()

state   regional_division  population        article_title  revision_id  \
0  Alabama  East South Central   5074296.0   Abbeville, Alabama       104730   
1  Alabama  East South Central   5074296.0  Adamsville, Alabama       104761   
2  Alabama  East South Central   5074296.0     Addison, Alabama       105188   
3  Alabama  East South Central   5074296.0       Akron, Alabama       104726   
4  Alabama  East South Central   5074296.0   Alabaster, Alabama       105109   

  article_quality  
0            Stub  
1            Stub  
2            Stub  
3            Stub  
4            Stub

In [10]:
df_quality_pop_division.to_csv('1K_wp_scored_city_articles_by_state.csv')

In [11]:
df_quality_pop_division

state   regional_division  population          article_title  \
0     Alabama  East South Central   5074296.0     Abbeville, Alabama   
1     Alabama  East South Central   5074296.0    Adamsville, Alabama   
2     Alabama  East South Central   5074296.0       Addison, Alabama   
3     Alabama  East South Central   5074296.0         Akron, Alabama   
4     Alabama  East South Central   5074296.0     Alabaster, Alabama   
..        ...                 ...         ...                    ...   
956  Arkansas  West South Central   3045637.0        Marie, Arkansas   
957  Arkansas  West South Central   3045637.0       Marion, Arkansas   
958  Arkansas  West South Central   3045637.0  Marked Tree, Arkansas   
959  Arkansas  West South Central   3045637.0    Marmaduke, Arkansas   
960  Arkansas  West South Central   3045637.0     Marshall, Arkansas   

     revision_id article_quality  
0         104730            Stub  
1         104761            Stub  
2         105188            Stub  
3         104726            Stub  
4         105109            Stub  
..           ...             ...  
956       107143           Start  
957       106945           Start  
958       107193           Start  
959       107005            Stub  
960       107248            Stub  

[961 rows x 6 columns]

# Step 4: Analysis

For the analysis, we will compute the total articles per capita for the states and regional divisions in our data and then in step 5 we will rank the results.

In [12]:
# Source: https://www.geeksforgeeks.org/python-pandas-dataframe-groupby/
# Source: https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-output-from-series-to-dataframe
# https://stackoverflow.com/questions/32751229/pandas-sum-by-groupby-but-exclude-certain-columns

df_state_article_counts = df_quality_pop_division.groupby(['state', 'population'], as_index=False)[['article_title']].agg('count')
df_state_article_counts

state  population  article_title
0   Alabama   5074296.0            443
1    Alaska    733583.0            142
2   Arizona   7359197.0             89
3  Arkansas   3045637.0            287

Now, let's do the math

In [13]:
df_articles_per_capita_state = df_state_article_counts
df_articles_per_capita_state['articles_per_capita_state'] = df_state_article_counts.article_title / df_state_article_counts.population

In [14]:
df_articles_per_capita_state

state  population  article_title  articles_per_capita_state
0   Alabama   5074296.0            443                   0.000087
1    Alaska    733583.0            142                   0.000194
2   Arizona   7359197.0             89                   0.000012
3  Arkansas   3045637.0            287                   0.000094

**Now we compute the total articles per capita for the regional divisions**

First we count the number of articles in each state

In [15]:
# Source: https://www.geeksforgeeks.org/python-pandas-dataframe-groupby/
# Source: https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-output-from-series-to-dataframe
# https://stackoverflow.com/questions/32751229/pandas-sum-by-groupby-but-exclude-certain-columns

df_division_article_counts = df_quality_pop_division.groupby(['regional_division', 'state', 'population'], as_index=False)[['article_title']].agg('count')
df_division_article_counts

regional_division     state  population  article_title
0  East South Central   Alabama   5074296.0            443
1            Mountain   Arizona   7359197.0             89
2             Pacific    Alaska    733583.0            142
3  West South Central  Arkansas   3045637.0            287

Then we add the population and article counts in each state to find the total population and article counts for each regional division.

In [16]:
# Source: https://www.geeksforgeeks.org/python-pandas-dataframe-groupby/
# Source: https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-output-from-series-to-dataframe
# https://stackoverflow.com/questions/32751229/pandas-sum-by-groupby-but-exclude-certain-columns

df_division_article_counts = df_division_article_counts.groupby(['regional_division'], as_index=False)[['population', 'article_title']].agg('sum')
df_division_article_counts

regional_division  population  article_title
0  East South Central   5074296.0            443
1            Mountain   7359197.0             89
2             Pacific    733583.0            142
3  West South Central   3045637.0            287

Now we can compute the per capita values

In [17]:
df_articles_per_capita_region = df_division_article_counts
df_articles_per_capita_region['articles_per_capita_region'] = df_division_article_counts.article_title / df_division_article_counts.population

In [18]:
df_articles_per_capita_region

regional_division  population  article_title  articles_per_capita_region
0  East South Central   5074296.0            443                    0.000087
1            Mountain   7359197.0             89                    0.000012
2             Pacific    733583.0            142                    0.000194
3  West South Central   3045637.0            287                    0.000094

**Article Quality:**

To determine the number of high-quality articles per state and region population (per capita) we start by determining whether or not each article is high-quality

In [19]:
df_high_quality = df_quality_pop_division

# I used the following sources to learn that np.where() existed and how to use it
# https://stackoverflow.com/questions/21702342/creating-a-new-column-based-on-if-elif-else-condition
# https://stackoverflow.com/questions/19913659/how-do-i-create-a-new-column-where-the-values-are-selected-based-on-existing-col?noredirect=1&lq=1
# https://numpy.org/doc/stable/reference/generated/numpy.where.html
# https://stackoverflow.com/questions/36921951/truth-value-of-a-series-is-ambiguous-use-a-empty-a-bool-a-item-a-any-o
# https://stackoverflow.com/questions/16343752/numpy-where-function-multiple-conditions


df_high_quality['is_high_quality'] = np.where((df_high_quality['article_quality'] == 'FA') 
                                                      | (df_high_quality['article_quality'] == 'GA'), 1, 0)
df_high_quality[df_high_quality['is_high_quality'] == 1]

state   regional_division  population    article_title  revision_id  \
38  Alabama  East South Central   5074296.0  Berlin, Alabama     23831085   

   article_quality  is_high_quality  
38              FA                1

We can see the high-quality results in the output directly above. Now, we can group by the states and regions to find the per capita high-quality articles

In [20]:
df_high_quality_count = df_high_quality.groupby(['regional_division', 'state', 'population'], as_index=False)['is_high_quality'].agg('sum')
df_high_quality_count

regional_division     state  population  is_high_quality
0  East South Central   Alabama   5074296.0                1
1            Mountain   Arizona   7359197.0                0
2             Pacific    Alaska    733583.0                0
3  West South Central  Arkansas   3045637.0                0

Next, we find the high-quality articles per capita for states

In [21]:
df_high_quality_state = df_high_quality_count.drop('regional_division', axis=1)
df_high_quality_state['high_quality_per_capita'] = df_high_quality_state['is_high_quality'] / df_high_quality_state['population']
df_high_quality_state

state  population  is_high_quality  high_quality_per_capita
0   Alabama   5074296.0                1             1.970717e-07
1   Arizona   7359197.0                0             0.000000e+00
2    Alaska    733583.0                0             0.000000e+00
3  Arkansas   3045637.0                0             0.000000e+00

Lastly, we find the high-quality articles per capita for regions. We start by grouping the data by region

In [22]:
df_high_quality_region = df_high_quality_count.groupby('regional_division', as_index=False)['population', 'is_high_quality'].agg('sum')
df_high_quality_region

/tmp/ipykernel_31505/3704886650.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_high_quality_region = df_high_quality_count.groupby('regional_division', as_index=False)['population', 'is_high_quality'].agg('sum')


regional_division  population  is_high_quality
0  East South Central   5074296.0                1
1            Mountain   7359197.0                0
2             Pacific    733583.0                0
3  West South Central   3045637.0                0

Now we compute the per capita values

In [23]:
df_high_quality_region['high_quality_per_capita'] = df_high_quality_region['is_high_quality'] / df_high_quality_region['population']
df_high_quality_region

regional_division  population  is_high_quality  high_quality_per_capita
0  East South Central   5074296.0                1             1.970717e-07
1            Mountain   7359197.0                0             0.000000e+00
2             Pacific    733583.0                0             0.000000e+00
3  West South Central   3045637.0                0             0.000000e+00

# Step 5: Results

According to the assignment instructions, I am asked to produce 6 tables depicting the results of this analysis. They are listed in bold below and are direct quotes from the assignment instructions.

**1.	Top 10 US states by coverage: The 10 US states with the highest total articles per capita (in descending order)**.

In [26]:
# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
table_1 = df_articles_per_capita_state.sort_values(by='articles_per_capita_state', axis=0, ascending=False, ignore_index=True)
table_1.head(10)

state  population  article_title  articles_per_capita_state
0    Alaska    733583.0            142                   0.000194
1  Arkansas   3045637.0            287                   0.000094
2   Alabama   5074296.0            443                   0.000087
3   Arizona   7359197.0             89                   0.000012

**2.	Bottom 10 US states by coverage: The 10 US states with the lowest total articles per capita (in ascending order)**.

In [27]:
# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
table_2 = df_articles_per_capita_state.sort_values(by='articles_per_capita_state', axis=0, ascending=True, ignore_index=True)
table_2.tail(10)

state  population  article_title  articles_per_capita_state
0   Arizona   7359197.0             89                   0.000012
1   Alabama   5074296.0            443                   0.000087
2  Arkansas   3045637.0            287                   0.000094
3    Alaska    733583.0            142                   0.000194

**3.	Top 10 US states by high quality: The 10 US states with the highest high quality articles per capita (in descending order)**.

In [29]:
# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
table_3 = df_high_quality_state.sort_values(by='high_quality_per_capita', axis=0, ascending=False, ignore_index=True)
table_3.head(10)

state  population  is_high_quality  high_quality_per_capita
0   Alabama   5074296.0                1             1.970717e-07
1   Arizona   7359197.0                0             0.000000e+00
2    Alaska    733583.0                0             0.000000e+00
3  Arkansas   3045637.0                0             0.000000e+00

**4.	Bottom 10 US states by high quality: The 10 US states with the lowest high quality articles per capita (in ascending order).**

In [30]:
# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
table_4 = df_high_quality_state.sort_values(by='high_quality_per_capita', axis=0, ascending=True, ignore_index=True)
table_4.tail(10)

state  population  is_high_quality  high_quality_per_capita
0   Arizona   7359197.0                0             0.000000e+00
1    Alaska    733583.0                0             0.000000e+00
2  Arkansas   3045637.0                0             0.000000e+00
3   Alabama   5074296.0                1             1.970717e-07

**5.	Census divisions by total coverage: A rank ordered list of US census divisions (in descending order) by total articles per capita.**

In [32]:
# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
table_5 = df_articles_per_capita_region.sort_values(by='articles_per_capita_region', axis=0, ascending=False, ignore_index=True)
table_5

regional_division  population  article_title  articles_per_capita_region
0             Pacific    733583.0            142                    0.000194
1  West South Central   3045637.0            287                    0.000094
2  East South Central   5074296.0            443                    0.000087
3            Mountain   7359197.0             89                    0.000012

**6.	Census divisions by high quality coverage: Rank ordered list of US census divisions (in descending order) by high quality articles per capita.**

In [34]:
# Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
table_6 = df_high_quality_region.sort_values(by='high_quality_per_capita', axis=0, ascending=False, ignore_index=True)
table_6

regional_division  population  is_high_quality  high_quality_per_capita
0  East South Central   5074296.0                1             1.970717e-07
1            Mountain   7359197.0                0             0.000000e+00
2             Pacific    733583.0                0             0.000000e+00
3  West South Central   3045637.0                0             0.000000e+00

# References:

- [1]. Homework 2 - Considering Bias in Data.docx